## 1. Check if dataset is valid

In [ ]:
import cv2
import imghdr
import os

In [ ]:
def check_images(data_dir_path):
    print("Checking images in", data_dir_path + "...")
    error = False
    image_exts = ["bmp", "jpeg", "jpg", "png"]
    
    for image in os.listdir(data_dir_path):
        image_path = os.path.join(data_dir_path, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                error = True
                print('Image not in ext list {}'.format(image_path))
                # os.remove(image_path)
        except Exception as e:
            error = True
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)
            
    if not error:
        print(data_dir_path, "is safe to use.")

In [ ]:
train_dir_path = "/kaggle/input/animals/animals/train"
val_dir_path = "/kaggle/input/animals/animals/val"
inf_dir_path = "/kaggle/input/animals/animals/inf"

In [ ]:
for image_class in os.listdir(train_dir_path): 
    check_images(os.path.join(train_dir_path, image_class))

In [ ]:
for image_class in os.listdir(val_dir_path):
    check_images(os.path.join(val_dir_path, image_class))

In [ ]:
check_images(inf_dir_path)

## 2. Load dataset

In [ ]:
import tensorflow as tf
import pathlib

In [ ]:
batch_size = 3
image_height = 200
image_width = 200

In [ ]:
train_dir = pathlib.Path(train_dir_path)
val_dir = pathlib.Path(val_dir_path)

In [ ]:
training_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

In [ ]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

## 3. Define the convolutional neural network

In [ ]:
from tensorflow.keras import layers

In [ ]:
num_classes = 5

In [ ]:
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(128, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

## 4. Prepare the model for training

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
logdir="logs"

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    histogram_freq=1,
    write_images=logdir,
    embeddings_data=training_data
)

In [ ]:
training_data = training_data.shuffle(buffer_size=1000, seed=42, reshuffle_each_iteration=True)

## 5. Train

In [ ]:
model.fit(
    training_data,
    validation_data=validation_data,
    epochs=20,
    callbacks=[tensorboard_callback]
)

## 6. Inference

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image_files = os.listdir(inf_dir_path)

In [ ]:
for file_name in image_files:
    file_path = os.path.join(inf_dir_path, file_name)
    image_to_predict = cv2.imread(file_path, cv2.IMREAD_COLOR)
    plt.imshow(cv2.cvtColor(image_to_predict, cv2.COLOR_BGR2RGB))
    plt.show()

    img_to_predict = np.expand_dims(cv2.resize(image_to_predict, (200, 200)), axis=0)

    predict = model.predict(img_to_predict)
    res = np.argmax(predict, axis=1)

    if res == 0:
        print("Cat")
    elif res == 1:
        print("Dog")
    elif res == 2:
        print("Elephant")
    elif res == 3:
        print("Horse")
    elif res == 4:
        print("Lion")